# Relation-specific Words Extraction

**Задача:** извлечь характерные слова для каждого типа отношений из корпуса абстрактов RuSERRC.

In [1]:
from collections import Counter
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
import pymorphy2 as pm2

### 1. Data Loading

В корпусе RuSERRC присутсвуют размеченные по типам отношений абстракты, но они записаны в обычном текстовом формате. Переведем их в более удобный формат .csv файла:

In [2]:
data_files = [
    '../data/ruserrc_dataset/ruserrc_v1/re_sci_data/train.txt',
    '../data/ruserrc_dataset/ruserrc_v1/re_sci_data/dev.txt',
    '../data/ruserrc_dataset/ruserrc_v2/ruserrc_v2_re_data/train.txt',
    '../data/ruserrc_dataset/ruserrc_v2/ruserrc_v2_re_data/dev.txt',
]

In [3]:
file_contents = []
uniques = set()
for file_name in tqdm(data_files, desc='Extracting source data'):
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            # removing NER tags
            clean_line = re.sub(r'\<e([0-9])\>(.+)\</e\1\>', r'\2', line).strip()

            # separating source text from its relation tag
            text, rel_tag = clean_line.rsplit(maxsplit=1)

            # counting each pair once
            if (text, rel_tag) in uniques:
                continue

            file_contents.append((text, rel_tag))
            uniques.add((text, rel_tag))

Extracting source data:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(file_contents, columns=['abstract', 'relation_tag'])
df

,abstract,relation_tag
0,В качестве дополнительного аппаратного обеспеч...,TOOL
1,Применение метода K - средних для идентификаци...,ISA
2,Задача решается методом частиц - в - ячейках.,TOOL
3,Применение методов нелинейной динамики для исс...,TOOL
4,В результате представлена архитектура вычислит...,USAGE
...,...,...
639,В соответствие с авторской методикой для каждо...,PART_OF
640,"Доказано, что любая точка этой окружности явля...",PART_OF
641,Приводится пример применения объекта - шлюза д...,TOOL
642,В процессе добычи урана методом подземного ск...,USAGE


In [14]:
df_directory = '../data/data_frames/ruserrc'

In [15]:
df.to_csv(f'{df_directory}/full.csv')

Для удобства анализа сохраним датафрейм и в виде подфреймов по каждому типу отношений:

In [16]:
for tag in df['relation_tag'].unique():
    type_df = df[df['relation_tag'] == tag].drop(columns=['relation_tag']).reset_index(drop=True)
    type_df.to_csv(f'{df_directory}/{tag}.csv')

### 2. Frequency Count

Подсчитаем для каждого типа отношений частотность слов:

In [17]:
df = pd.read_csv(f'{df_directory}/full.csv', index_col=0)
df['relation_tag'].value_counts()

relation_tag
USAGE       295
ISA         114
TOOL         75
PART_OF      74
SYNONYMS     49
CAUSE        28
COMPARE       9
Name: count, dtype: int64

In [18]:
relation_types = list(df['relation_tag'].unique())
relation_types

['TOOL', 'ISA', 'USAGE', 'PART_OF', 'CAUSE', 'SYNONYMS', 'COMPARE']

In [19]:
lemmatizer = pm2.MorphAnalyzer(lang='ru')

In [20]:
def get_or_cache_token_normal_form(lemmatizer, token, cache):
    """Extracts token's normal form from cache, or adds it to the latter if not found."""
    if token in cache:
        token_normal_form = cache[token]
    else:
        token_normal_form = lemmatizer.parse(token)[0].normal_form
        cache[token] = token_normal_form

    return token_normal_form

Будем также учитывать предлоги, тире и двоеточие. Удалим первых из списка стоп-слов: 

In [23]:
stop_words = stopwords.words('russian')
prepositions = {
    'в', 'без', 'до', 'для', 'за', 'через', 'над', 'по', 'из', 'у',
    'около', 'под', 'о', 'про', 'на', 'к', 'перед', 'при', 'с', 'между',
}

stop_words = set(stop_words)
stop_words -= prepositions

In [76]:
pm2_cache = {}
rel_freqs_dfs = []
for rel_type in tqdm(relation_types, desc='Counting word frequencies per relation type'):
    # merging all abtracts related to one relation type
    text = df[df['relation_tag'] == rel_type]['abstract'].str.cat(sep=' ').lower()

    # filtering out tokens
    tokens = [token for token in word_tokenize(text, language='russian') if token.isalpha() or token in {':', '–'}]

    # normalizing tokens
    normal_tokens = [get_or_cache_token_normal_form(lemmatizer, token, pm2_cache) for token in tokens]

    # removing stopwords
    final_tokens = [token for token in normal_tokens if token not in stop_words]

    # counting frequencies
    rel_freqs_dfs.append(pd.DataFrame(Counter(final_tokens).most_common(), columns=['token', 'TF']))

    # removing infrequent words
    rel_freqs_dfs[-1] = rel_freqs_dfs[-1][rel_freqs_dfs[-1]['TF'] >= 2]

    # initializing probabilities
    rel_freqs_dfs[-1]['relation_proba'] = 0
    rel_freqs_dfs[-1]['overall_proba'] = 0

    # counting relational probabilities
    relation_sentences = sent_tokenize(df[df['relation_tag'] == rel_type]['abstract'].str.cat(sep=' '), language='russian')
    for sentence in relation_sentences:
        sentence_tokens = [token for token in word_tokenize(sentence, language='russian') if token.isalpha() or token in {':', '–'}]
        sentence_tokens = [get_or_cache_token_normal_form(lemmatizer, token, pm2_cache) for token in sentence_tokens]
        sentence_tokens = set([token for token in sentence_tokens if token not in stop_words])

        for word in rel_freqs_dfs[-1]['token']:
            if word in sentence_tokens:
                rel_freqs_dfs[-1].loc[rel_freqs_dfs[-1]['token'] == word, 'relation_proba'] += 1
    rel_freqs_dfs[-1]['relation_proba'] = rel_freqs_dfs[-1]['relation_proba'].div(len(relation_sentences))

    # counting overall probabilities
    overall_sentences = sent_tokenize(df['abstract'].str.cat(sep=' '), language='russian')
    for sentence in overall_sentences:
        sentence_tokens = [token for token in word_tokenize(sentence, language='russian') if token.isalpha() or token in {':', '–'}]
        sentence_tokens = [get_or_cache_token_normal_form(lemmatizer, token, pm2_cache) for token in sentence_tokens]
        sentence_tokens = set([token for token in sentence_tokens if token not in stop_words])

        for word in rel_freqs_dfs[-1]['token']:
            if word in sentence_tokens:
                rel_freqs_dfs[-1].loc[rel_freqs_dfs[-1]['token'] == word, 'overall_proba'] += 1
    rel_freqs_dfs[-1]['overall_proba'] = rel_freqs_dfs[-1]['overall_proba'].div(len(overall_sentences))

    # removing very rare (in overall corpus) words
    rel_freqs_dfs[-1] = rel_freqs_dfs[-1][rel_freqs_dfs[-1]['overall_proba'] >= 1e-2]

    # counting probability ratio
    rel_freqs_dfs[-1]['proba_ratio'] = rel_freqs_dfs[-1]['relation_proba'] / rel_freqs_dfs[-1]['overall_proba']

    # sorting by probability ratio (descending order)
    rel_freqs_dfs[-1].sort_values(by=['proba_ratio', 'TF'], inplace=True, ascending=[False, False], ignore_index=True)

Counting word frequencies per relation type:   0%|          | 0/7 [00:00<?, ?it/s]

### 3. Relation-specific Words Filtering

Выведем характерные слова по каждому типу отношений:

##### TOOL

In [85]:
rel_freqs_dfs[0].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,проводить,4,0.051282,0.010671,4.805861
1,патоген,4,0.051282,0.013720,3.737892
2,профилирование,4,0.038462,0.010671,3.604396
3,филогенетический,3,0.038462,0.010671,3.604396
4,исследовать,4,0.051282,0.015244,3.364103
5,понятие,4,0.051282,0.015244,3.364103
6,хаос,4,0.038462,0.012195,3.153846
7,форма,4,0.038462,0.012195,3.153846
8,предназначить,3,0.038462,0.012195,3.153846
9,дерево,4,0.051282,0.016768,3.058275


##### ISA

In [86]:
rel_freqs_dfs[1].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,написать,4,0.035714,0.010671,3.346939
1,выбрать,5,0.044643,0.013720,3.253968
2,характеристика,7,0.062500,0.022866,2.733333
3,показатель,8,0.053571,0.019817,2.703297
4,свойство,5,0.044643,0.016768,2.662338
5,–,18,0.133929,0.050305,2.662338
6,поверхность,5,0.035714,0.013720,2.603175
7,оператор,5,0.035714,0.013720,2.603175
8,обладать,4,0.035714,0.013720,2.603175
9,содержать,4,0.035714,0.013720,2.603175


##### USAGE

In [87]:
rel_freqs_dfs[2].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,базироваться,9,0.029801,0.013720,2.172185
1,совместный,6,0.019868,0.010671,1.861873
2,сценарий,9,0.026490,0.015244,1.737748
3,изображение,10,0.029801,0.018293,1.629139
4,визуализация,7,0.019868,0.012195,1.629139
5,выполняться,6,0.019868,0.012195,1.629139
6,выявление,6,0.019868,0.012195,1.629139
7,основать,16,0.052980,0.033537,1.579771
8,автор,10,0.033113,0.021341,1.551561
9,java,7,0.016556,0.010671,1.551561


##### PARTOF

In [88]:
rel_freqs_dfs[3].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,семантический,7,0.092105,0.019817,4.647773
1,компонент,7,0.092105,0.024390,3.776316
2,состоять,9,0.118421,0.032012,3.699248
3,частность,3,0.039474,0.010671,3.699248
4,необходимый,3,0.039474,0.010671,3.699248
5,основной,5,0.065789,0.018293,3.596491
6,модуль,11,0.118421,0.033537,3.531100
7,элемент,4,0.052632,0.015244,3.452632
8,нефтегазовый,4,0.039474,0.012195,3.236842
9,включать,4,0.052632,0.016768,3.138756


##### CAUSE

In [89]:
rel_freqs_dfs[4].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,скорость,4,0.153846,0.013720,11.213675
1,правило,3,0.115385,0.013720,8.410256
2,возможный,2,0.076923,0.010671,7.208791
3,проектный,2,0.076923,0.010671,7.208791
4,развитие,3,0.115385,0.016768,6.881119
5,механизм,2,0.076923,0.012195,6.307692
6,влияние,2,0.076923,0.012195,6.307692
7,продукт,2,0.076923,0.012195,6.307692
8,форма,2,0.076923,0.012195,6.307692
9,русский,2,0.076923,0.012195,6.307692


##### SYNONYMS

In [90]:
rel_freqs_dfs[5].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,величина,3,0.057692,0.010671,5.406593
1,гибридный,3,0.057692,0.010671,5.406593
2,называть,3,0.057692,0.012195,4.730769
3,соответствие,3,0.057692,0.012195,4.730769
4,рекомендательный,3,0.038462,0.010671,3.604396
5,протокол,3,0.038462,0.010671,3.604396
6,дифференциальный,3,0.038462,0.010671,3.604396
7,через,2,0.038462,0.010671,3.604396
8,подобный,2,0.038462,0.010671,3.604396
9,написать,2,0.038462,0.010671,3.604396


##### COMPARE

In [91]:
rel_freqs_dfs[6].head(50)

,token,TF,relation_proba,overall_proba,proba_ratio
0,сравнительный,3,0.3,0.018293,16.400000
1,веб,3,0.2,0.015244,13.120000
2,реляционный,2,0.2,0.016768,11.927273
3,сравнение,2,0.2,0.016768,11.927273
4,опыт,2,0.1,0.010671,9.371429
5,сайт,2,0.1,0.012195,8.200000
6,открытый,2,0.1,0.012195,8.200000
7,образовательный,2,0.1,0.013720,7.288889
8,архитектура,3,0.2,0.028963,6.905263
9,провести,2,0.2,0.032012,6.247619


Сохраним статистики в отдельную директорию:

In [92]:
save_directory = '../data/data_frames/rel_specific_words_probability'

for rel_name, data in zip(relation_types, rel_freqs_dfs):
    data.to_csv(f'{save_directory}/{rel_name}.csv')

### 4. Conclusion

(**NEW**) У нового подхода есть явный недостаток: при подсчете пропорции **намного больший вес имеет знаменатель** (потому что часто он << числителя) -> т.о., редкие слова имеют неоправданно высокий score.

Изначально, вообще все слова в топе были с tf=1, что точно не то, что нам нужно, ведь это очень редкие слова.

Как пытался улучшить:
1. ограничение tf >= 10. Очень жесткое ограничение, выдача становилась адекватнее, но редкие отношения (COMPARE) вообще без слов оставались;
2. tf >= 2 и overall_proba >= 0.01 (это вероятность по всему корпусу). Этот вариант использую сейчас, выдача побогаче, но качество выдачи хуже.

Обе модификации не дают существенно более адекватной выдачи. Может, стоит **поменять изначальную формулу пропорции**? Как снизить влияние знаменателя? По идее, в идеале **числитель и знаменатель отношения должны вносить равный вклад** в score.

#### 4.1 Notes

- в качестве датасета были выбраны **train и test** части **двух версий** RuSERRC (и сохранены для удобства в один датафрейм);
- при предобработке исходных абстрактов **сохранялись 2 вида пунктуации**: тире и двоеточие. Я заметил, что оба они появляются чаще обычного в предложениях типа ISA (а просто двоеточие еще и в типе PARTOF). Да, они не слова, но они тоже характерны для конкретных типов отношений - в каком-то смысле, они описывают структуру повествования ("x - это ..." или "x состоит из: ..."), поэтому я решил их ради исключения оставить;
- кое-что аналогичное заметил и с предлогами, но они в целом очень частотные по всем типам отношений, поэтому их все выкинул, они лишь мешают сбору статистики;
- в некоторых предложениях **тип отношения очень неочевидный** (даже для человека), наверное это объясняет, почему в топ пробираются и не характерные слова. Как с этим бороться? Если принципиально нужно, можно в целом и ручками почистить, благо что выжимка небольшая совсем вышла. Также есть предположение, что увеличение размера корпуса поможет частично с проблемой шумов (чем боьше данных, тем более частотными становятся интересующие нас слова).

#### 4.2 Improvements

- найти англоязычный датасет и извлечь из него характерные слова (на примете - SciERC, но надо проверить, есть ли там размеченные предложения по типам отношений);
- (?) придумать еще более хитрую фильтрацию, чтобы поменьше нехарактерных слов сохранялось. Но на маленьком датасете это вряд ли даст прирост качества;
- (?) расширить датасет, тогда и характерные слова будут лучше выделяться среди шумов.